### Importing Required Libraries

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

### BI-LSTM Model

In [ ]:
model = Sequential([
    Embedding(input_dim=10000, output_dim=128),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.5),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Model: "sequential"

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

### Train the Model

In [20]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=16)

Epoch 1/5
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 231s 103ms/step - accuracy: 0.8190 - loss: 0.4102 - val_accuracy: 0.8625 - val_loss: 0.3308
Epoch 2/5
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 219s 100ms/step - accuracy: 0.8941 - loss: 0.2789 - val_accuracy: 0.8824 - val_loss: 0.3064
Epoch 3/5
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 214s 98ms/step - accuracy: 0.9228 - loss: 0.2079 - val_accuracy: 0.8670 - val_loss: 0.3261
Epoch 4/5
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 204s 93ms/step - accuracy: 0.9411 - loss: 0.1587 - val_accuracy: 0.8755 - val_loss: 0.3355
Epoch 5/5
2188/2188 ━━━━━━━━━━━━━━━━━━━━ 209s 95ms/step - accuracy: 0.9578 - loss: 0.1202 - val_accuracy: 0.8734 - val_loss: 0.3840


### Evaluate the Model

In [22]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

469/469 ━━━━━━━━━━━━━━━━━━━━ 13s 27ms/step - accuracy: 0.8734 - loss: 0.3840
Test Accuracy: 0.87


## Summary of Results:
#### - Model Architecture: Embedding → BI-LSTM → Dropout → Dense layers
#### - Dataset Size: ~50,000 samples
#### - Data Split: 70% training, 30% testing
#### - Training Setup: 5 epochs, batch size = 16, validation_split = 0.3
#### - Accuracy: **87.34%**